# A Fast Parallel Stochastic Gradient Method for Matrix Factorization in Shared Memory Systems

Ce projet repose sur l'article "A Fast Parallel Stochastic Gradient Method for Matrix Factorization in Shared Memory Systems".

In [1]:
import pyquickhelper, pyensae
import random
import numpy as np
import pandas as pd

### Création d'une matrice sparse pour tester l'algorithme de factorisation matricielle

Cette matrice est aléatoire avec 100000 observations non nulles. Elle permet d'implémenter notre algorithme sur un petit jeu de données.

In [2]:
from scipy import sparse as sp

In [3]:
nbUsers = 300

In [4]:
nbFilms = 50

##### Initialisation de la matrice de données

In [6]:
a = sp.rand(nbUsers,nbFilms,0.1)

In [8]:
data=np.array([a.row, a.col, a.data])

In [9]:
data=np.transpose(data)

##### Initialisation des matrices P et Q

In [12]:
K=3

In [13]:
p=np.random.uniform(0,1,((nbUsers*K)))
row_p=[x for x in range(0,nbUsers) for j in range(0,K)]
col_int=[i for j in range(0,nbUsers) for i in range(0,K)]
Psparse = sp.coo_matrix((p,(row_p,col_int)))

In [15]:
q=np.random.uniform(0,1,((nbFilms*K)))
row_q=[x for x in range(0,nbFilms) for j in range(0,K)]
col_int=[i for j in range(0,nbFilms) for i in range(0,K)]
Qsparse = sp.coo_matrix((q,(row_q,col_int)))

### Implémentation de l'algorithme sous python sans parallélisation

Pour exécuter l'algorithme sans parallélisation, nous parcourons chaque observation et nous mettons à jour la valeur de P et Q selon la formule issue de la minimisation.

In [16]:
N=len(a.row) #Nombre

In [17]:
nb=5*N #Nombre d'itérations

In [20]:
V=np.random.choice(range(N),nb, p=np.repeat(1/N,N)) #On fixe des probabilités de répétitions de chaque point égales.
#Pour N grand, tous les points sont en moyenne parcourus le même nombre de fois. 
#On peut aussi fixer un nb de répétitions fixes égal pour chaque valeur en exécutant 
# [x for x in range(0,nb) for j in range(0,N)] puis mélanger.

In [18]:
def matrix_factorization (R, Psparse, Qsparse, K, V, gamma =0.02 , lambd =0.02):
    P=np.array([Psparse.row, Psparse.col, Psparse.data])
    P=np.transpose(P)
    Q=np.array([Qsparse.row, Qsparse.col, Qsparse.data])
    Q=np.transpose(Q)
    for l in V:
        i=R[l][0]
        j=R[l][1]
        eij=R[l][2]-np.dot(P[Psparse.row==i,2],Q[Qsparse.row==j,2])
        for k in range(K):
            P[(Psparse.row==i) & (Psparse.col==k),2] = P[(Psparse.row==i) & (Psparse.col==k),2] + gamma * (2 * eij * Q[(Qsparse.row==j) & (Qsparse.col==k),2] - lambd * P[(Psparse.row==i) & (Psparse.col==k),2])
            Q[(Qsparse.row==j) & (Qsparse.col==k),2] = Q[(Qsparse.row==j) & (Qsparse.col==k),2] + gamma * (2 * eij * P[(Psparse.row==i) & (Psparse.col==k),2] - lambd * Q[(Qsparse.row==j) & (Qsparse.col==k),2])
    return P, Q

In [21]:
matrix_factorization(R=data,Psparse=Psparse,Qsparse=Qsparse,K=3,V=V)

(array([[  0.00000000e+00,   0.00000000e+00,   2.49452360e-01],
        [  0.00000000e+00,   1.00000000e+00,   6.75605988e-01],
        [  0.00000000e+00,   2.00000000e+00,   2.54645105e-01],
        ..., 
        [  2.99000000e+02,   0.00000000e+00,   4.51672033e-01],
        [  2.99000000e+02,   1.00000000e+00,   7.75548268e-01],
        [  2.99000000e+02,   2.00000000e+00,   7.16348854e-01]]),
 array([[  0.00000000e+00,   0.00000000e+00,   8.07530619e-01],
        [  0.00000000e+00,   1.00000000e+00,   5.15311408e-01],
        [  0.00000000e+00,   2.00000000e+00,   2.40838840e-02],
        [  1.00000000e+00,   0.00000000e+00,   9.15505081e-02],
        [  1.00000000e+00,   1.00000000e+00,   5.19989177e-01],
        [  1.00000000e+00,   2.00000000e+00,   6.07525307e-01],
        [  2.00000000e+00,   0.00000000e+00,   6.75946499e-01],
        [  2.00000000e+00,   1.00000000e+00,   1.61076686e-02],
        [  2.00000000e+00,   2.00000000e+00,   1.44981072e-01],
        [  3.00000000e+0

Ou une alternative si nous souhaitons le même format de matrice sparse en entrée et en sortie de la fonction :

In [31]:
def matrix_factorization (R, Psparse, Qsparse, K, V, gamma =0.02 , lambd =0.02):
    for l in V:
        i=R[l][0]
        j=R[l][1]
        eij = R[l][2] - np.dot(Psparse.data[Psparse.row==i],Qsparse.data[Qsparse.row==j])
        for k in range (K):
            Psparse.data[(Psparse.row==i) & (Psparse.col==k)] = Psparse.data[(Psparse.row==i) & (Psparse.col==k)] + gamma * (2 * eij * Qsparse.data[(Qsparse.row==j) & (Qsparse.col==k)] - lambd * Psparse.data[(Psparse.row==i) & (Psparse.col==k)])
            Qsparse.data[(Qsparse.row==j) & (Qsparse.col==k)] = Qsparse.data[(Qsparse.row==j) & (Qsparse.col==k)] + gamma * (2 * eij * Psparse.data[(Psparse.row==i) & (Psparse.col==k)] - lambd * Qsparse.data[(Qsparse.row==j) & (Qsparse.col==k)])
    return Psparse, Qsparse

In [32]:
P2, Q2 = matrix_factorization(R=data,Psparse=Psparse,Qsparse=Qsparse,K=3,V=V)

In [33]:
print(P2)

  (0, 0)	0.249452360179
  (0, 1)	0.675605988293
  (0, 2)	0.25464510508
  (1, 0)	0.445049632916
  (1, 1)	0.326428663976
  (1, 2)	0.143108390523
  (2, 0)	0.533689681654
  (2, 1)	0.856682740704
  (2, 2)	0.0963479880937
  (3, 0)	0.773602326886
  (3, 1)	0.138103972902
  (3, 2)	0.624317041969
  (4, 0)	0.354837934384
  (4, 1)	0.488653162265
  (4, 2)	0.494888027263
  (5, 0)	0.347192935275
  (5, 1)	0.7646108432
  (5, 2)	0.34140975413
  (6, 0)	0.149467367564
  (6, 1)	0.0475103342701
  (6, 2)	0.79036024681
  (7, 0)	0.796879995339
  (7, 1)	0.318420555256
  (7, 2)	0.505683672816
  (8, 0)	0.630127568626
  :	:
  (291, 2)	0.378462129701
  (292, 0)	0.42063130038
  (292, 1)	0.313805185772
  (292, 2)	0.528522996381
  (293, 0)	0.371745983116
  (293, 1)	0.429641763085
  (293, 2)	0.840938556409
  (294, 0)	0.547345607064
  (294, 1)	0.262887833228
  (294, 2)	0.19968851471
  (295, 0)	0.618355914758
  (295, 1)	0.504488401521
  (295, 2)	0.478501836848
  (296, 0)	0.926275024984
  (296, 1)	0.128671234028
  (296, 2

### Fonction pour mélanger la matrice à factoriser afin de répartir aléatoirement les valeurs non nulles

### Ecriture de la fonction (kernel) appliquée à chaque bloc. Cette fonction est appliquée à chaque thread.

In [44]:
def noyau(bloc, blocp, blocq, K, gamma =0.02 , lambd =0.02):
    for l in range (len(bloc.row)):
        i=bloc.row[l]
        j=bloc.col[l]
        if bloc.data[(bloc.row==i) & (bloc.col==j)] > 0:
            eij = bloc.data[(bloc.row==i) & (bloc.col==j)] - np.dot(blocp.data[blocp.row==i],blocq.data[blocq.row==j])
            for k in range (K):
                blocp.data[(blocp.row==i) & (blocp.col==k)] = blocp.data[(blocp.row==i) & (blocp.col==k)] + gamma * (2 * eij * blocq.data[(blocq.row==j) & (blocq.col==k)] - lambd * blocp.data[(blocp.row==i) & (blocp.col==k)])
                blocq.data[(blocq.row==j) & (blocq.col==k)] = blocq.data[(blocq.row==j) & (blocq.col==k)] + gamma * (2 * eij * blocp.data[(blocp.row==i) & (blocp.col==k)] - lambd * blocq.data[(blocq.row==j) & (blocq.col==k)])
    return blocp, blocq

In [46]:
blocp, blocq = noyau(bloc=a,blocp=Psparse, blocq=Qsparse, K=3)